# Building Agents

시작하기 전에 [이 슬라이드](https://docs.google.com/presentation/d/13c0L1CQWAL7fuCXakOqjkvoodfynPJI4Hw_4H76okVU/edit?usp=sharing)를 참고하여 배경 지식을 확인하세요!

이메일 어시스턴트를 처음부터 구축해 보겠습니다. 
1) 랭그래프로 에이전트 아키텍처 설계
2) 랭스미스로 테스트
3) 휴먼 인 더 루프(Human-in-the-loop), 
4) 메모리 순으로 진행합니다. 

이 다이어그램은 각 구성 요소가 어떻게 결합되는지 보여줍니다:

![overview-img](assets/overview.png)

## Resources

- Notebook Reference: [agent.ipynb](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/agent.ipynb)
- For LangSmith Studio: [src/email_assistant](https://github.com/langchain-ai/agents-from-scratch/tree/main/src/email_assistant)
- Slides: [Building Ambient Agents with LangGraph - Building Agents & Evaluations.pdf](https://files.cdn.thinkific.com/file_uploads/967498/attachments/5f6/a6b/958/Building_Ambient_Agents_with_LangGraph_-_Building_Agents___Evaluations.pdf)



## 환경 설정

In [ ]:
import os
import sys
from pathlib import Path

# 현재 디렉토리를 Python 경로에 추가
current_dir = os.path.join(Path.cwd())
if str(current_dir) not in sys.path:
    sys.path.insert(0, str(current_dir))

In [35]:
from dotenv import load_dotenv


load_dotenv("../../.env", override=True)

False

## 도구 정의

이메일 어시스턴트가 사용할 간단한 도구들을 `@tool` 데코레이터로 정의합니다.

In [36]:
from datetime import datetime

from langchain_core.tools import tool
from pydantic import BaseModel


@tool
def write_email(to: str, subject: str, content: str) -> str:
    """이메일을 작성하여 보냅니다."""
    # 임시 응답 - 실제 앱에서는 이메일을 발송합니다
    return f"{to}에게 발송된 이메일, 제목: '{subject}', 내용: {content}"


@tool
def schedule_meeting(
    attendees: list[str],
    subject: str,
    duration_minutes: int,
    preferred_day: datetime,
    start_time: int,
) -> str:
    """캘린더에 회의를 예약합니다."""
    # 임시 응답 - 실제 앱에서는 캘린더와 일정을 확인합니다
    date_str = preferred_day.strftime("%A, %B %d, %Y")
    return f"'{subject}' 회의가 {date_str} {start_time}에 {duration_minutes}분 동안 {len(attendees)}명의 참석자와 함께 예정되어 있습니다"


@tool
def check_calendar_availability(day: str) -> str:
    """특정 날짜의 캘린더 가능 시간을 확인합니다."""
    # 임시 응답 - 실제 앱에서는 실제 캘린더를 확인합니다
    return f"{day}의 가능한 시간: 오전 9:00, 오후 2:00, 오후 4:00"


@tool
class Done(BaseModel):
    """이메일이 발송되었습니다."""

    done: bool

## 이메일 어시스턴트 구축하기

라우터와 에이전트를 결합하여 이메일 어시스턴트를 구축합니다.

![agent_workflow_img](./assets/email_workflow.png)

### 라우터(Router)

라우팅 단계는 분류 결정을 처리합니다.

분류 라우터는 분류 결정에만 집중하고, 에이전트는 응답에만 집중합니다.

#### 상태(State)

에이전트를 구축할 때는 시간에 따라 추적하고자 하는 정보를 고려하는 것이 중요합니다. 이번 프로젝트에서는 LangGraph의 사전 구축된 [`MessagesState`](https://langchain-ai.github.io/langgraph/concepts/low_level/#messagesstate)를 사용합니다. `MessagesState`를 확장하여 `email_input`와 `classification_decision` 키를 추가하는 사용자 정의 `State` 객체를 정의합니다.

In [37]:
from typing import Literal

from langgraph.graph import MessagesState


class State(MessagesState):
    email_input: dict
    classification_decision: Literal["ignore", "respond", "notify"]

#### 분류 노드

분류 라우팅 로직을 포함하는 함수를 정의합니다. 
이를 위해 [`structured_outputs`](https://python.langchain.com/docs/concepts/structured_outputs/)을 사용합니다. 
`Pydantic`은 타입 힌트와 유효성 검사를 제공하기 때문에 구조화된 출력 스키마를 정의하는 데 유용합니다. Pydantic 모델의 설명(description)은 JSON 스키마의 일부로 LLM에 전달되어 출력 변환에 대한 정보를 제공합니다.

In [39]:
from src.email_assistant.utils import parse_email, format_email_markdown

ModuleNotFoundError: No module named 'html2text'